In [72]:
import bindata as bnd
import pandas as pd
import numpy as np
capsules = pd.read_csv('../capsules_with_freq.csv', sep=';')
drug_names_unique = capsules[['Drug_1', 'Drug_2', 'Drug_3', 'Drug_4', 'Drug_5', 'Drug_6', 'Drug_7', 'Drug_8']].stack().unique()
# normalize capsules['exact_exposure_count] to 0, 100
capsules['exact_exposure_count'] = capsules['exact_exposure_count'].replace('<100', 100)
capsules['exact_exposure_count'] = capsules['exact_exposure_count'].replace('1', 100)
capsules['exact_exposure_count'] = capsules['exact_exposure_count'].astype(int)
capsules['exact_exposure_count'] = capsules['exact_exposure_count'] / (capsules['exact_exposure_count']).max() * 20000
capsules['exact_exposure_count'] = capsules['exact_exposure_count'].astype(int) + 1

# Restructure the data (columns are drugs and rows are patients, values are 0 or 1)
data = []
for i, row in capsules.iterrows():
    patient_data = []
    for drug in drug_names_unique:
        patient_data.append(int(drug in row.values))
    
    data.append(patient_data)
    # data.extend([patient_data] * row['exact_exposure_count'])

index = [f"comb{i}" for i in range(len(data))]
data = pd.DataFrame(data, columns=drug_names_unique, dtype=int, index=index)
data.head()
data.shape

feature_probs = np.mean(data, axis=0)
# feature_probs = np.round(feature_probs * 20) / 20 # round to 0.05 step
# feature_probs = np.ceil(feature_probs * 1e12) / 1e12
# feature_probs = np.array([0.07] * 40)

correlation_matrix = np.corrcoef(data, rowvar=False) # correlation, same as (np.cov(X,Y)[0,1]/(np.std(X,ddof=1)*np.std(Y,ddof=1)))
correlation_matrix = np.ceil(correlation_matrix * 1e12) / 1e12
# store feat_probs and correlation_matrix csv
np.savetxt('feat_probs_smoother.csv', feature_probs, delimiter=',')
np.savetxt('correlation_matrix.csv', correlation_matrix, delimiter=',')
# ====================================================================== 
def is_pos_def(x):
    return np.all(np.linalg.eigvals(x) > 0)
# ====================================================================== https://research.wu.ac.at/ws/portalfiles/portal/18952613/document.pdf
from statsmodels.stats.correlation_tools import cov_nearest
from scipy.stats import multivariate_normal, norm

commonprob = bnd.bincorr2commonprob(margprob=feature_probs, 
                                        bincorr=correlation_matrix)
# commonprob2sigma
sigma = bnd.commonprob2sigma(commonprob)
if not is_pos_def(sigma):
    sigma = cov_nearest(sigma)

µ, Σ = norm.ppf(feature_probs), sigma
mvn = multivariate_normal(µ, Σ, allow_singular=True)
sample = mvn.rvs(size=10000) > 0

/Users/karina/Library/Python/3.9/lib/python/site-packages/bindata/check_commonprob.py:81: UserWarning:

simulvals provided is not computed to a sufficient resolution
to resolve the common probabilies provided in commonprob.


INTERPOLATION IS GOING TO BE USED.

Consider computing simulvals on a finer grid.
The current resolution is on these points:
[(0, 0), (0, 0.001), (0, 0.01), (0, np.float64(0.05)), (0, np.float64(0.1)), (0, np.float64(0.15)), (0, np.float64(0.2)), (0, np.float64(0.25)), (0, np.float64(0.3)), (0, np.float64(0.35)), (0, np.float64(0.4)), (0, np.float64(0.45)), (0, np.float64(0.5)), (0, np.float64(0.55)), (0, np.float64(0.6)), (0, np.float64(0.65)), (0, np.float64(0.7)), (0, np.float64(0.75)), (0, np.float64(0.8)), (0, np.float64(0.85)), (0, np.float64(0.9)), (0, np.float64(0.95)), (0, 0.99), (0, 0.999), (0, 1), (0.001, 0.001), (0.001, 0.01), (0.001, np.float64(0.05)), (0.001, np.float64(0.1)), (0.001, np.float64(0.15)), (0.001, np.float64(0.2)), (0.001, np.float64(0.

In [68]:
from data_generator.main import generate_multivariate_binary, filter_rows

nsamples = 1000000
sample, mu, sigma = generate_multivariate_binary(40, nsamples, feature_probs, correlation_matrix)
sample = filter_rows(sample, mu, sigma, nsamples)

In [69]:
# group by sum 
sample = sample.astype(int)
sampledf = pd.DataFrame(data, columns=drug_names_unique)
sampledf['sum'] = sampledf.sum(axis=1)
print(sampledf['sum'].value_counts().sort_index())

sampledf = pd.DataFrame(sample, columns=drug_names_unique)
sampledf['sum'] = sampledf.sum(axis=1)
print(sampledf['sum'].value_counts().sort_index())

sample_new = sample[np.sum(sample, axis=1) > 2]
sample_new = sample_new[np.sum(sample_new, axis=1) <= 7]
sample_new.shape

sum
3    352760
4     29011
5      1735
6       104
7        26
8         6
Name: count, dtype: int64
sum
3    409657
4    284670
5    168245
6     91462
7     45966
Name: count, dtype: int64


(1000000, 40)

In [174]:
# import data_generator_EP as dg
# from data_generator_EP import gen_bin_ep

# sample = gen_bin_ep(100000, feature_probs, correlation_matrix)

In [73]:
sample.astype(int)

import plotly.express as px
corr_with_zero_diag = np.corrcoef(sample, rowvar=False) - np.eye(sample.shape[1])
fig = px.imshow(corr_with_zero_diag, x=drug_names_unique, y=drug_names_unique, zmin=-1, zmax=1, color_continuous_scale='RdBu')
fig.show()

corr_with_zero_diag = np.corrcoef(data, rowvar=False) - np.eye(sample.shape[1])
fig = px.imshow(corr_with_zero_diag, x=drug_names_unique, y=drug_names_unique, zmin=-1, zmax=1, color_continuous_scale='RdBu')
fig.show()

fig = px.bar(x=drug_names_unique, y=np.mean(data, axis=0))
fig.update_xaxes(categoryorder='total descending')
fig.show()
fig = px.bar(x=drug_names_unique, y=np.mean(sample, axis=0))
# fig.update_xaxes(categoryorder='total descending')
fig.show()


In [104]:
corr = np.array([[1., -0.25, -0.0625],
                 [-0.25,   1.,  0.25],
                 [-0.0625, 0.25, 1.]])
commonprob = bnd.bincorr2commonprob(margprob=[0.05, 0.05, 0.15], 
                                        bincorr=corr)
print(commonprob)
sigma = bnd.commonprob2sigma(commonprob)
X = bnd.rmvbin(margprob=np.diag(commonprob), 
                   commonprob=commonprob, N=100)
print(X.mean(0))
print(np.corrcoef(X, rowvar=False))

[[0.05       0.         0.00263613]
 [0.         0.05       0.02695548]
 [0.00263613 0.02695548 0.15      ]]
[0.06 0.03 0.12]
[[ 1.         -0.04443104 -0.09329556]
 [-0.04443104  1.         -0.06494176]
 [-0.09329556 -0.06494176  1.        ]]


In [53]:
import pandas as pd
import numpy as np

dataset_3comb = pd.read_csv('../doi_10_5061_dryad_sm847__v20181016/db_drugs_3s.tsv', sep='\t')
dataset_4comb = pd.read_csv('../doi_10_5061_dryad_sm847__v20181016/db_drugs_4s.tsv', sep='\t')
dataset_5comb = pd.read_csv('../doi_10_5061_dryad_sm847__v20181016/db_drugs_5s.tsv', sep='\t')
dataset_3comb['drug_names'] = dataset_3comb[['drug_name_A', 'drug_name_B', 'drug_name_C']].values.tolist()
dataset_4comb['drug_names'] = dataset_4comb[['drug_name_A', 'drug_name_B', 'drug_name_C', 'drug_name_D']].values.tolist()
dataset_5comb['drug_names'] = dataset_5comb[['drug_name_A', 'drug_name_B', 'drug_name_C', 'drug_name_D', 'drug_name_E']].values.tolist()

dataset = pd.concat([dataset_3comb, dataset_4comb, dataset_5comb])

drug_names_unique_2 = np.unique(np.concatenate(dataset['drug_names'].values))
drug_names_unique_lowcase = np.char.lower(drug_names_unique.astype(str))
drug_names_unique_lowcase = np.where(drug_names_unique_lowcase == 'fluticasone nasal', 'fluticasone', drug_names_unique_lowcase)
drug_names_unique_lowcase = np.where(drug_names_unique_lowcase == 'levothyroxine', 'thyroxine', drug_names_unique_lowcase)

print(dataset.shape)

# filter out every row that contain at least one drug that is not in drug_names_unique_lowcase
dataset = dataset[dataset['drug_names'].apply(lambda x: all([drug in drug_names_unique_lowcase for drug in x]))]
dataset = dataset[['drug_names', 'exact_exposure_count']]
dataset['exact_exposure_count'] = dataset['exact_exposure_count'].replace('<100', 100)

capsules_with_exactexposue_1 = capsules[capsules['exact_exposure_count'] == 1]
merged_dataset = pd.concat([dataset, capsules_with_exactexposue_1[['drug_names', 'exact_exposure_count']]])
# everything that contained 1 exact_exposure_count -> 100 exact_exposure_count
merged_dataset['exact_exposure_count'] = merged_dataset['exact_exposure_count'].replace(1, 100)
merged_dataset['exact_exposure_count'] = merged_dataset['exact_exposure_count'].astype(int)
# normalize capsules['exact_exposure_count] to 0, 1000
merged_dataset['exact_exposure_count'] = merged_dataset['exact_exposure_count'] / (merged_dataset['exact_exposure_count']).max() * 10000
merged_dataset['exact_exposure_count'] = merged_dataset['exact_exposure_count'].astype(int) + 1

print(merged_dataset.shape)

(151746, 13)
(19924, 2)


In [54]:
# Restructure the data (columns are drugs and rows are patients, values are 0 or 1)
data_full = []
for i, row in merged_dataset.iterrows():
    patient_data = []
    for drug in drug_names_unique_lowcase:
        patient_data.append(int(drug in row['drug_names']))
    
    # data.append(patient_data)
    data_full.extend([patient_data] * row['exact_exposure_count'])

index = [f"comb{i}" for i in range(len(data_full))]
data_full = pd.DataFrame(data_full, columns=drug_names_unique_lowcase, dtype=int, index=index)
print(data_full.shape)
data_full.head()

(392730, 40)


,carvedilol,esomeprazole,thyroxine,tramadol,amlodipine,metoprolol,omeprazole,simvastatin,tamsulosin,alendronate,...,zolpidem,allopurinol,citalopram,prednisone,rosuvastatin,valsartan,ibuprofen,fexofenadine,montelukast,fluticasone
comb0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
comb1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
comb2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
comb3,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
comb4,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# sampledf = pd.DataFrame(data_full, columns=drug_names_unique)
# data_full['sum'] = data_full.sum(axis=1)
# print(data_full.sum(axis=1).value_counts().sort_index())

3    267838
4    104257
5     20635
Name: count, dtype: int64


In [199]:
corr_with_zero_diag = np.corrcoef(data_full, rowvar=False) - np.eye(data_full.shape[1])
fig = px.imshow(corr_with_zero_diag, x=drug_names_unique, y=drug_names_unique, zmin=-0.4, zmax=0.4, color_continuous_scale='RdBu')
fig.show()

fig = px.bar(x=drug_names_unique, y=data_full.mean(0))
fig.update_xaxes(categoryorder='total descending')
fig.show()

In [203]:
data_full.mean(0)

carvedilol             0.036713
esomeprazole           0.039470
thyroxine              0.175072
tramadol               0.011647
amlodipine             0.201033
metoprolol             0.208141
omeprazole             0.092534
simvastatin            0.254053
tamsulosin             0.024570
alendronate            0.024837
atorvastatin           0.188990
lisinopril             0.429613
losartan               0.127521
sertraline             0.039559
metformin              0.178038
pioglitazone           0.032088
pravastatin            0.057258
furosemide             0.037071
potassium chloride     0.000183
hydrochlorothiazide    0.471345
atenolol               0.097741
warfarin               0.029569
aspirin                0.000979
lovastatin             0.034764
albuterol              0.022863
glipizide              0.046965
ranitidine             0.010344
gabapentin             0.016322
insulin glargine       0.017192
clopidogrel            0.064972
zolpidem               0.030785
allopuri

In [201]:
feature_probs = np.mean(data_full, axis=0)
# feature_probs = np.round(feature_probs * 20) / 20 # round to 0.05 step
correlation_matrix = np.corrcoef(data_full, rowvar=False) # correlation, same as (np.cov(X,Y)[0,1]/(np.std(X,ddof=1)*np.std(Y,ddof=1)))
# correlation_matrix = np.round(correlation_matrix, 3)
# np.savetxt('feat_probs.csv', feature_probs, delimiter=',')
# np.savetxt('correlation_matrix.csv', correlation_matrix, delimiter=',')
# ======================================================================
commonprob = bnd.bincorr2commonprob(margprob=feature_probs, 
                                        bincorr=correlation_matrix)
# commonprob2sigma
sigma = bnd.commonprob2sigma(commonprob)
µ, Σ = norm.ppf(feature_probs), sigma
# find nearest positive semidefinite
Σ = cov_nearest(Σ)
# sample = multivariate_normal.rvs(µ, Σ, size=N)
mvn = multivariate_normal(µ, Σ, allow_singular=True)
sample = bnd.ra2ba(mvn.rvs(size=100000))

In [202]:
corr_with_zero_diag = np.corrcoef(sample, rowvar=False) - np.eye(sample.shape[1])
fig = px.imshow(corr_with_zero_diag, x=drug_names_unique, y=drug_names_unique, zmin=-0.4, zmax=0.4, color_continuous_scale='RdBu')
fig.show()

fig = px.bar(x=drug_names_unique, y=sample.mean(0))
fig.update_xaxes(categoryorder='total descending')
fig.show()